In [1]:
import numpy as np

from scipy.stats import norm, t

## Generalised Welch-Satterthwaite Equation

The following function implementes the generalised Welch-Satterthwaite equation with more than two $s^2$ abd $n$, as specified in Satterthwaite (1946) and Welch (1947):

$$\nu = \frac{\left(\sum_{g \in G}\psi_g \right)^2}{\sum_{g \in G}\frac{\psi^2_g}{n_g - 1}},$$
where $\psi_g = \frac{s^2_g}{n_g}$.

It takes the following arguments:
- `s_squared`: $[s_g^2]_{g\in G}$, the sample variance of a group.
- `n`: $[n_g]_{g\in G}$, the size of a group.

...and returns value of $\nu$.

In [2]:
def generalised_welch_satterthwaite(s_squared, n):
    """
    Calculate the generalised Welch-Satterthwaite equation with
    more than two s^2 and n, specified in Satterthwaite (1946) 
    and Welch (1947).
    :param s_squared: 1-d array-like containing all s^2_i values
    :param n: 1-d array-like containing all n values (the index
              should match with that in s_squared)
    :return: The result of the generalised Welch-Satterthwaite
             equation (aka the approximate degrees of freedom
             for the generalised Welch's t-statistic)
    """
    
    numerator_acc = 0
    denominator = 0
    
    for i in range(0, len(s_squared), 1):
        numerator_acc += s_squared[i] / n[i]
        denominator += np.power(s_squared[i], 2) / \
                       ((n[i] - 1) * n[i] * n[i])
            
    return np.power(numerator_acc, 2) / denominator

In [3]:
generalised_welch_satterthwaite([0.25, 0.1, 0.2, 2], [100, 100, 100, 100])

156.53434650455929

## Minimum Sample Size - Equal Size in All Groups

The following function gives a solution of $n_{\min}$ to the following inequality:
$$n_{\min} > \left(\frac{t_{\nu, 1-\alpha} - t_{\nu, 1-\pi_{\min}}}{\theta}\right)^2 \sum_{g \in G} s_g^2.$$

It takes the following arguments:
- `s_squared`: $[s_g^2]_{g\in G}$, the sample variance of a group.
- `theta`: $\theta$, the intended effect size one wanted to detect.
- `significance_level`: $\alpha$, the significance level (usually 0.05 or 5%).
- `min_power`: $\pi_{\min}$, the minimum power (usually 0.8 or 80%)

... and returns $n_{\min}$, the minimum sample size required of a group, assuming the size of all groups are equal.

In [4]:
def norm_quantile_diff(significance_level, min_power):
    return np.power((norm.ppf(1 - significance_level) - norm.ppf(1 - min_power)), 2)

def t_quantile_diff(significance_level, min_power, nu):
    return np.power((t.ppf(1 - significance_level, nu) - t.ppf(1 - min_power, nu)), 2)
    
def get_min_sample_sizes(rhs_func, **kwargs):
    """
    A minimum sample size estimation procedure which iteratively evaluates
    n_min, nu (and hence the quantile difference) 
    """
    s_squared = kwargs['s_squared']
    
    rhs_normal_quantile = rhs_func(**kwargs)
    n_min = np.ceil(rhs_normal_quantile)
    nu = generalised_welch_satterthwaite(
        s_squared, np.repeat(n_min, np.array(s_squared).shape[0]))
    rhs_t_quantile = rhs_func(**kwargs, nu=nu)

    while (n_min <= rhs_t_quantile):
        n_min = np.ceil(rhs_t_quantile)
        nu = generalised_welch_satterthwaite(
            s_squared, np.repeat(n_min, np.array(s_squared).shape[0]))
        rhs_t_quantile = rhs_func(**kwargs, nu=nu)
                                             
    return n_min

def get_min_sample_sizes_equal(s_squared, theta, significance_level=0.05, min_power=0.8):
    def rhs_value_equal(s_squared, theta, 
                        significance_level, min_power, nu=None):
        if nu is None:
            quantile_diff = norm_quantile_diff(significance_level, min_power)
        else:
            quantile_diff = t_quantile_diff(significance_level, min_power, nu)
        
        return quantile_diff * np.sum(s_squared) / np.power(theta, 2)
    
    return(
        get_min_sample_sizes(
            rhs_value_equal, s_squared=s_squared, theta=theta,
            significance_level=significance_level, min_power=min_power))

In [5]:
get_min_sample_sizes_equal(
    s_squared=[0.25, 0.25, 0.25, 0.25], theta=0.005, 
    significance_level=0.05, min_power=0.8)

247303.0

## Minimum Sample Size - Fixed Sample Size Ratio Between Groups

The following function gives a solution of $n_{\min}$ to the following inequality:
$$n_{\min} > \left(\frac{t_{\nu, 1-\alpha} - t_{\nu, 1-\pi_{\min}}}{\theta}\right)^2 \sum_{g \in G} \frac{k_1}{k_g} s_g^2.$$

Here $n_{\min}$ refers to the number of samples required by the **first** group. To calculate the number of samples required in other groups, multiply $n_{\min}$ by $\frac{k_g}{k_1}$.

It takes the following arguments:
- `s_squared`: $[s_g^2]_{g\in G}$, the sample variance of a group.
- `ratio`: $[k_g]_{g\in G}$, the ratio between groups.
- `theta`: $\theta$, the intended effect size one wanted to detect.
- `significance_level`: $\alpha$, the significance level (usually 0.05 or 5%).
- `min_power`: $\pi_{\min}$, the minimum power (usually 0.8 or 80%)

In [6]:
def get_min_sample_sizes_ratioed(s_squared, ratio, theta,
                                 significance_level=0.05, min_power=0.8):
    
    def rhs_value_ratioed(s_squared, ratio, theta, 
                          significance_level, min_power, nu=None):
        if nu is None:
            quantile_diff = norm_quantile_diff(significance_level, min_power)
        else:
            quantile_diff = t_quantile_diff(significance_level, min_power, nu)
        
        return(
            quantile_diff * 
            np.sum(np.array(s_squared) * ratio[0] / np.array(ratio)) / \
            np.power(theta, 2))
    
    assert (len(ratio) > 0), "Please provide the ratio of at least one group"
            
    return(
        get_min_sample_sizes(
            rhs_value_ratioed, s_squared=s_squared, ratio=ratio, theta=theta,
            significance_level=significance_level, min_power=min_power))

In [7]:
get_min_sample_sizes_ratioed(
    s_squared=[0.25, 0.25, 0.25, 0.25], ratio=[1, 1, 5, 5], 
    theta=0.005, significance_level=0.05, min_power=0.8)

148382.0